Progetto finale IDS Davide Cifarelli S4699037

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F #cross entropy loss
import torchtext
from torchtext.legacy.data import Field, BucketIterator, Iterator
from torchtext.legacy import data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from tokenize import tokenize, untokenize   #python tokenizer
import spacy
import io
import random
import math
import time  
from tqdm import tqdm
from google.colab import files  #dataset
from google.colab import drive
from nltk.translate.bleu_score import sentence_bleu        #bleuscore
from nltk.translate.bleu_score import SmoothingFunction

In [2]:
uploaded = files.upload()

Saving dataset.txt to dataset.txt


In [3]:
%set_env CUDA_LAUNCH_BLOCKING = 1 # disabilita lanci del kernel asincroni ed e utile per il debugging

env: CUDA_LAUNCH_BLOCKING=1 # disabilita lanci del kernel asincroni ed e utile per il debugging


Nel nostro dataset le righe che contengono le domande iniziano sempre con '#'.
Tutto cio' che e' compreso tra due domande e' una risposta.

In [4]:
f = open("dataset.txt", "r")
filelines = f.readlines()
totQA = []                                       # lista contente coppie di domande/risposte del nostro dataset
QuestionAndAnswer = None
for line in filelines:
  if line[0] == "#":                           # ogni linea contente # e' una linea contente una domanda
    if QuestionAndAnswer:
      QuestionAndAnswer['solution'] = ''.join(QuestionAndAnswer['solution'])    
      totQA.append(QuestionAndAnswer)                           # aggiungo la coppia domanda risposta
    QuestionAndAnswer = {"question": None, "solution": []}
    QuestionAndAnswer['question'] = line[1:]                  # aggiungo la domanda alla lista
  else:
    QuestionAndAnswer["solution"].append(line)                # aggiungo tutte le linee dove e' scritta la soluzione



Creo una funzione che funzioni da tokenizer per la sintassi di python usando la libreria ufficiale di python. [Documentazione](https://docs.python.org/3/library/tokenize.html) 


In [5]:
def tokenize_code(code_str):  
    code_tokens = list(tokenize(io.BytesIO(code_str.encode('utf-8')).readline))
    tokenized_output = []    #lista contenente tutti i token della frase in input
    for i in range(0, len(code_tokens)):
        tokenized_output.append((code_tokens[i].type, code_tokens[i].string))
    return tokenized_output                                      

Test di prova con una risposta del dataset

In [6]:
sample = tokenize_code(totQA[1]['solution'])

In [7]:
print(sample)

[(57, 'utf-8'), (1, 'def'), (1, 'add_two_numbers'), (53, '('), (1, 'num1'), (53, ','), (1, 'num2'), (53, ')'), (53, ':'), (4, '\n'), (5, '    '), (1, 'sum'), (53, '='), (1, 'num1'), (53, '+'), (1, 'num2'), (4, '\n'), (1, 'return'), (1, 'sum'), (4, '\n'), (56, '\n'), (56, '\n'), (6, ''), (0, '')]


In [8]:
print(untokenize(sample).decode('utf-8'))

def add_two_numbers (num1 ,num2 ):
    sum =num1 +num2 
    return sum 





Siccome il dataset e' abbastanza piccolo cerco di aumentarlo. Durante la tokenizzazione del codice in python applico una maschera su certe variabili, in 
maniera randomica per evitare che il modello non si concentri sui nomi delle variabili, ma sulla sintassi. Allo stesso tempo creo una lista di keyword che non devono essere considerate come variabili e quindi non vanno modificate.


In [9]:
def augment_tokenize_code(code_str, mask_factor=0.3):

    var_masked = {} # Dizionario che memorizza le variabili mascherate

    # alcune parole che non dovrebbero essere trattate come normali variabili e
    # quindi devono non devono essere modificate mentre aumentiamo il nostro dataset

    skip_words = ['range', 'enumerate', 'print', 'ord', 'int', 'float', 'zip'
                 'char', 'list', 'dict', 'tuple', 'set', 'len', 'sum', 'min',
                 'max','False', 'None', 'True', 'and', 'as', 'assert', 'async',
                 'await', 'break','class', 'continue', 'def', 'del', 'elif',
                 'else', 'except', 'finally', 'for','from', 'global', 'if',
                 'import', 'in', 'is', 'lambda', 'nonlocal', 'not', 'or',
                 'pass', 'raise', 'return', 'try', 'while', 'with', 'yield']
    
    

    counter = 1 #var counter
    code_tokens = list(tokenize(io.BytesIO(code_str.encode('utf-8')).readline)) # tokenizzo la string in input
    tokenized_output = []  # questa sara la nuova lista di tokens con le variabili mascherate

    i=0
    while i in  range(0, len(code_tokens)):
      if code_tokens[i].type == 1 and code_tokens[i].string not in skip_words:
        
        if i>0 and code_tokens[i-1].string in ['def', '.', 'import', 'raise', 'except', 'class']: # evito di mascherare le parole successive a queste
          skip_words.append(code_tokens[i].string)                                                 # aggiungo alla lista di skip le parole         
          tokenized_output.append((code_tokens[i].type, code_tokens[i].string))
        elif code_tokens[i].string in var_masked:                                                   # se gia mascherata la variabile
          tokenized_output.append((code_tokens[i].type, var_masked[code_tokens[i].string]))
        elif random.uniform(0, 1) > 1-mask_factor:                                                  # maschero in maniera randomica le variabili
          var_masked[code_tokens[i].string] = 'var_' + str(counter)                             # variabili mascherate var_1,var_2...
          counter+=1
          tokenized_output.append((code_tokens[i].type, var_masked[code_tokens[i].string]))       #aggiungo le var mascherate
        else:
          skip_words.append(code_tokens[i].string)                                                 # se no aggiungo la variabile alla lista degli skip
          tokenized_output.append((code_tokens[i].type, code_tokens[i].string))
      
      else:
        tokenized_output.append((code_tokens[i].type, code_tokens[i].string))

      i+=1
    
    return tokenized_output

Test di prova con una risposta del dataset

In [10]:
sample = augment_tokenize_code(totQA[1]['solution'])
print(sample)

[(57, 'utf-8'), (1, 'def'), (1, 'add_two_numbers'), (53, '('), (1, 'num1'), (53, ','), (1, 'num2'), (53, ')'), (53, ':'), (4, '\n'), (5, '    '), (1, 'sum'), (53, '='), (1, 'num1'), (53, '+'), (1, 'num2'), (4, '\n'), (1, 'return'), (1, 'sum'), (4, '\n'), (56, '\n'), (56, '\n'), (6, ''), (0, '')]


In [11]:
print(untokenize(sample).decode('utf-8'))

def add_two_numbers (num1 ,num2 ):
    sum =num1 +num2 
    return sum 





In [12]:
codeQA_df = pd.DataFrame(totQA)   #creo il dataframe del dataset

for i in range(0,codeQA_df.shape[0]):
  codeQA_df['question'].iloc[i] = codeQA_df['question'].iloc[i].lstrip('0123456789.- ') #alcune domande avevano un numero all'inizio quindi lo rimuovo

codeQA_df.head()

,question,solution
0,write a python program to add two numbers \n,num1 = 1.5\nnum2 = 6.3\nsum = num1 + num2\npri...
1,write a python function to add two user provid...,"def add_two_numbers(num1, num2):\n sum = nu..."
2,write a program to find and print the largest ...,\nnum1 = 10\nnum2 = 12\nnum3 = 14\nif (num1 >=...
3,write a program to find and print the smallest...,num1 = 10\nnum2 = 12\nnum3 = 14\nif (num1 <= n...
4,Write a python function to merge two given lis...,"def merge_lists(l1, l2):\n return l1 + l2\n..."


Divido il mio dataset:
Training (85%), validazione(7.5%), testing (7.5%)

In [13]:
np.random.seed(0)

#msk1,msk2 sono liste di var booleane o true o false che corrispondono a ogni elemento del 
#dataframe code questions and answers, in corrispondenza dei valori true la riga del dataframe sara'
#aggiunta al nuovo dataframe (train o validazione o test)

msk1 = np.random.rand(len(codeQA_df)) < 0.85 

train_df = codeQA_df[msk1]
val_and_test_df = codeQA_df[~msk1]

msk2 = np.random.rand(len(val_and_test_df)) < 0.5

val_df = val_and_test_df[msk2]
test_df = val_and_test_df[~msk2]


In [14]:
#Seeds per cuda

SEED = 123
random.seed(SEED)                           # utilizzo seed per avere un primo valore random in base al seed scelto (per avere behaviour deterministico durante il training)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [15]:
# Qui uso i Torchtext Fields per costruire il vocabolario per il problema di apprendimento da sequenza a sequenza.
# Per capire l'inizio e la fine di una stringa si usano due token rispettivamente <sos> e <eos>

Input = data.Field(tokenize = 'spacy', init_token='<sos>', eos_token='<eos>', lower=True)

Output = data.Field(tokenize = augment_tokenize_code, init_token='<sos>', eos_token='<eos>', 
                    lower=False)

fields = [('Input', Input),('Output', Output)]

Estendo il vocabolario di 100x


In [16]:
train_ex = []
val_ex = []

train_expansion = 100
for j in range(train_expansion):
  for i in range(train_df.shape[0]):
      try:
          ex = data.Example.fromlist([train_df.question[i], train_df.solution[i]], fields)
          train_ex.append(ex)
      except:
          pass

for i in range(val_df.shape[0]):
    try:
        ex = data.Example.fromlist([val_df.question[i], val_df.solution[i]], fields)
        val_ex.append(ex)
    except:
        pass       

train_data = data.Dataset(train_ex, fields)   
valid_data =  data.Dataset(val_ex, fields)

Input.build_vocab(train_data, min_freq = 0)    #costruisco il vocabolario
Output.build_vocab(train_data, min_freq = 0)

Utilizzo cuda per sfruttare la GPU

In [17]:
USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")
device

device(type='cuda')

L'architettura transformer puo' essere suddivisa in 3 parti:
1. Un codificatore che codifica una sequenza di input in un vettore
2. Un meccanismo di attenzione che fa focalizzare sulla parte piu importante della frase. Usato sia nel codificatore che nel decodificatore.
3. Un decodificatore che decodifica il vettore in output


Converto i token della sequenza di parole in inglese in embedding('tok_embedding') di lunghezza 'hid_dim'. Bisogna taggare ogni token con il suo indice di posizione per preservare le informazioni sequenziali. Creo un tensore di indici ('pos') e lo converto in un embedding('pos_embedding') di lunghezza 'hid_dim'. Questo è combinato con i token embeddings della sequenza per creare il tensore di input src. Il tensore src viene passato attraverso una serie di Encoder Layers.

In [18]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hid_dim, n_layers, n_heads, pf_dim,dropout, device, max_length = 1000):
        super().__init__()

        self.device = device #cpu o cuda

        self.tok_embedding = nn.Embedding(input_dim, hid_dim)      #eseguo l'embedding dei token
        self.pos_embedding = nn.Embedding(max_length, hid_dim)     #eseguo l'embedding della posizione del token nella frase
        
        self.layers = nn.ModuleList([EncoderLayer(hid_dim, n_heads, pf_dim,dropout,device) for _ in range(n_layers)])
        
        self.dropout = nn.Dropout(dropout)   #dropout alcuni nodi vengono ignorati per evitare overfitting
        
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device) #radice quadrata della dimensione del vettore
        
    def forward(self, src, src_mask):
        
        batch_size = src.shape[0]
        src_len = src.shape[1]

        position = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)        
        src = self.dropout((self.tok_embedding(src) * self.scale) + self.pos_embedding(position))
        #applico il dropout

        for layer in self.layers:     #l'input tensor src viene passato attraverso i layer
            src = layer(src, src_mask)
            
        return src

Un EncoderLayer è l'elemento di base dell'encoder del transformer. Il tensore src insieme alla sua maschera src (src_mask) vengono inviati in un multi-head self-attention per aiutare il nostro modello a concentrarsi sugli aspetti piu importanti del tensore src. L'output dell'operazione di attenzione è combinato con il tensore src e normalizzato per evitare gradient vanishing/exploding  durante l'allenamento. Questo output combinato viene inviato ad un FeedForward Layer.



In [19]:
class EncoderLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, pf_dim,  dropout, device):
        super().__init__()
        
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)          
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout,device)
        self.feedforward = FeedforwardLayer(hid_dim, pf_dim, dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src, src_mask):
                        
        _src, _ = self.self_attention(src, src, src, src_mask)     #self attention       
        src = self.self_attn_layer_norm(src + self.dropout(_src))  #dropout  e normalizzazione        
        _src = self.feedforward(src)                               #feedforward        
        src = self.ff_layer_norm(src + self.dropout(_src))         #dropout  e normalizzazione 
        
        return src

L'attenzione è un meccanismo che consente a un modello di concentrarsi sulle parti piu rilevanti della sequenza di input in base al compito da svolgere.

Quando più canali (heads) di attenzione vengono applicati in parallelo a una singola fonte, si parla di attenzione multi-head. Essa aumenta la capacità di apprendimento del modello e quindi porta a risultati migliori.

In [20]:
class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, dropout,device):
        super().__init__()
        
        assert hid_dim % n_heads == 0 #True? Se si proseguo, hid_dim lunghezza dei token embedded
        #Utilizzando multihead dividiamo N batch di vettori embedded per N heads

        self.hid_dim = hid_dim
        self.n_heads = n_heads
        self.head_dim = hid_dim // n_heads  #arrotondo
        
        #v,q,k copie esatte del src tensor
        self.fc_q = nn.Linear(hid_dim, hid_dim)  #singolo layer feedforward
        self.fc_k = nn.Linear(hid_dim, hid_dim)
        self.fc_v = nn.Linear(hid_dim, hid_dim)       
        self.fc_o = nn.Linear(hid_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)
        #radice quadrata della dimensione di K
        
    def forward(self, query, key, value, mask = None):
        
        batch_size = query.shape[0]
                
        Q = self.fc_q(query)
        K = self.fc_k(key)
        V = self.fc_v(value)
                
        #con view creo delle copie del tensore base Q,K,V
        Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)  # con permute calcolo la matrice trasposta
        K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
                
        
        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale
        # qui moltiplico Q per la trasposta di K e divido il tutto per la dimensione di K
        
        if mask is not None:
            energy = energy.masked_fill(mask == 0, -1e10)    #se e' presente una maschera quando trovo 0 sostituisco con -inf
        
        attention = torch.softmax(energy, dim = -1)    #calcolo la softmax del prodotto ricavato precendentemente
                
        x = torch.matmul(self.dropout(attention), V)   #infine per ottenere l'attentione moltiplico per V
        
        x = x.permute(0, 2, 1, 3).contiguous()         #creo una copia del tensore
        
        x = x.view(batch_size, -1, self.hid_dim)       
        
        x = self.fc_o(x)
        
        return x, attention

Un FeedForward Layer prende l'input combinato e lo elabora ulteriormente utilizzando due layer completamente connessi e una funzione di attivazione Relu tra di loro. Questo in combinazione con l'src è l'output finale di un EncoderLayer. Questo processo si ripete per ogni blocco EncoderLayer.


In [21]:
class FeedforwardLayer(nn.Module):
    def __init__(self, hid_dim, pf_dim, dropout):
        super().__init__()

        #2 operatori lineari e un dropout e una ReLU nel feedforward layer        
        self.fc_1 = nn.Linear(hid_dim, pf_dim)
        self.fc_2 = nn.Linear(pf_dim, hid_dim)        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        x = self.dropout(torch.relu(self.fc_1(x)))        #dropout+ReLU
        x = self.fc_2(x)
        
        return x

L' architettura di un Decoder è molto simile a quella dell'encoder con le differenze significative derivanti dalla presenza di due input diversi, la sequenza target (trg) e il vettore uscente dall'encoder. Proprio come abbiamo avuto un blocco EncoderLayer per Encoder, avremo un DecoderLayer che accetta come input la combinazione della sequenza di token dell'output di codice(tok_embedding) e gli indici posizionali per questi token (pos_embedding). E come accennato in precedenza, l'output dell'encoder funge anche da input per DecoderLayer.


In [22]:
class Decoder(nn.Module):
    def __init__(self, output_dim, hid_dim, n_layers, n_heads, pf_dim, dropout, device, max_length = 10000):
        super().__init__()
        
        self.device = device #cuda o cpu

        #ora nel decoder la sequenza di input e' l'output ovvero il codice in python
        self.tok_embedding = nn.Embedding(output_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        
        self.layers = nn.ModuleList([DecoderLayer(hid_dim, n_heads, pf_dim, dropout,device) for _ in range(n_layers)])
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
        
    def forward(self, trg, enc_src, trg_mask, src_mask): # qui funziona come per l'encoder
                
        batch_size = trg.shape[0]
        trg_len = trg.shape[1]
        
        position = torch.arange(0, trg_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)

        trg = self.dropout((self.tok_embedding(trg) * self.scale) + self.pos_embedding(position))
        
        for layer in self.layers:
            trg, attention = layer(trg, enc_src, trg_mask, src_mask) 
        
        output = self.fc_out(trg)
            
        return output, attention

 Ogni DecoderLayer prevede due operazioni di attenzione:
1. Self-attention sul trg.
2. Multi-head attention che utilizza trg come vettore query Q e le uscite dell'encoder come vettore chiave K e valore V.

La presenza di un multihead attention aggiuntivo differenzia il DecoderLayer da un EncoderLayer.


In [23]:
class DecoderLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, pf_dim, dropout,device):
        super().__init__()
        
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.enc_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout,device)
        self.encoder_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout,device)
        self.feedforward = FeedforwardLayer(hid_dim, pf_dim, dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, trg, enc_src, trg_mask, src_mask):
        
        _trg, _ = self.self_attention(trg, trg, trg, trg_mask)                      #self attention operator
        
        trg = self.self_attn_layer_norm(trg + self.dropout(_trg))                   #dropout e normalizzazione layer
            
        _trg, attention = self.encoder_attention(trg, enc_src, enc_src, src_mask)   #attention dell'encoder-decoder
        
        trg = self.enc_attn_layer_norm(trg + self.dropout(_trg))                    #dropout e normalizzazione layer
                    
        _trg = self.feedforward(trg)                                                #feedforward
        
        trg = self.ff_layer_norm(trg + self.dropout(_trg))                          #attention dell'encoder-decoder
        
        #la differenzia sostanziale qui con l'encoder e' che vi e' un multihead attention operator che considera sia l'output
        #dell'encoder sia l'output del self attention del decoder 
        
        return trg, attention

La classe principale che implementa un transformer:


In [24]:
class Transformer(nn.Module):
    def __init__(self, encoder, decoder, src_pad_idx, trg_pad_idx,device):
        super().__init__()        
        self.device=device
        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
                
    def make_src_mask(self, src): # questa maschera nasconde gli input nulli tra i tokens della frase in inglese
        
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2) #lo stesso tensore con una nuova dimensione in posizione 1 e 2
        return src_mask
    
    def make_trg_mask(self, trg): # questa maschera nasconde le parole successive dell'output per evitare che influenzino la scelta delle prossime parole in output
        
        trg_pad_mask = (trg != self.trg_pad_idx).unsqueeze(1).unsqueeze(2)
        trg_len = trg.shape[1]       
        trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len), device = self.device)).bool() # tril resituisce il triangolo inferiore sinistro della matrice e mette a zero gli altri valori
        trg_mask = trg_pad_mask & trg_sub_mask                                                 # ones restituisce una matrice di 1
        
        return trg_mask

    def forward(self, src, trg):
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)        
        enc_src = self.encoder(src, src_mask)                
        output, attention = self.decoder(trg, enc_src, trg_mask, src_mask)      
        
        return output, attention

In [25]:
INPUT_DIM = len(Input.vocab)
OUTPUT_DIM = len(Output.vocab)
HID_DIM = 256
ENC_LAYERS = 3
DEC_LAYERS = 3
ENC_HEADS = 16
DEC_HEADS = 16
ENC_PF_DIM = 512
DEC_PF_DIM = 512
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1

enc = Encoder(INPUT_DIM, HID_DIM, ENC_LAYERS, ENC_HEADS, ENC_PF_DIM, ENC_DROPOUT, device)
dec = Decoder(OUTPUT_DIM, HID_DIM, DEC_LAYERS, DEC_HEADS, DEC_PF_DIM, DEC_DROPOUT, device)

SRC_PAD_IDX = Input.vocab.stoi[Input.pad_token]
TRG_PAD_IDX = Output.vocab.stoi[Output.pad_token]

model = Transformer(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device).to(device)

def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data) #inizializzo i weights


model.apply(initialize_weights);
LEARNING_RATE = 0.0004  # learning rate ottimale per i problemi di NLP tra 3e-4 e 5e-4
optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)

Ho utilizzato la data augmentation per mascherare le variabili. Ciò significa che il modello può prevedere diversi valori per una variabile e tutti sono corretti se le previsioni sono coerenti. Ciò significa che le variabili usate nel training non sono per forza le uniche giuste e quindi ha più senso trattarle come corrette con probabilità 1- smooth_eps e errate altrimenti. Per questo utilizzo l'implementazione del CrossEntropyLoss con label smoothing


In [26]:
#codice sorgente cross entropy loss e label smoothing presentato nel paper Rethinking the Inception Architecture for Computer Vision: https://arxiv.org/abs/1512.00567 
'''https://github.com/eladhoffer/utils.pytorch/blob/master/cross_entropy.py'''

class CrossEntropyLoss(nn.CrossEntropyLoss):
    def __init__(self, weight=None, ignore_index=-100, reduction='mean', smooth_eps=None, smooth_dist=None, from_logits=True):
        super(CrossEntropyLoss, self).__init__(weight=weight,ignore_index=ignore_index, reduction=reduction)
        self.smooth_eps = smooth_eps
        self.smooth_dist = smooth_dist
        self.from_logits = from_logits

    def forward(self, input, target, smooth_dist=None):
        if smooth_dist is None:
            smooth_dist = self.smooth_dist
        return cross_entropy(input, target, weight=self.weight, ignore_index=self.ignore_index,
                             reduction=self.reduction, smooth_eps=self.smooth_eps,
                             smooth_dist=smooth_dist, from_logits=self.from_logits)


def cross_entropy(inputs, target, weight=None, ignore_index=-100, reduction='mean',
                  smooth_eps=None, smooth_dist=None, from_logits=True):
    smooth_eps = smooth_eps or 0

    #log-liklihood
    if _is_long(target) and smooth_eps == 0:
        if from_logits:
            return F.cross_entropy(inputs, target, weight, ignore_index=ignore_index, reduction=reduction)  
        else:
            return F.nll_loss(inputs, target, weight, ignore_index=ignore_index, reduction=reduction)   # perdita negativa log-likelihood

    if from_logits:
        lsm = F.log_softmax(inputs, dim=-1)
    else:
        lsm = inputs

    masked_indices = None
    num_classes = inputs.size(-1) 

    if _is_long(target) and ignore_index >= 0:
        masked_indices = target.eq(ignore_index)

    if smooth_eps > 0 and smooth_dist is not None:
        if _is_long(target):
            target = onehot(target, num_classes).type_as(inputs)
        if smooth_dist.dim() < target.dim():
            smooth_dist = smooth_dist.unsqueeze(0)
        target.lerp_(smooth_dist, smooth_eps)

    if weight is not None:
        lsm = lsm * weight.unsqueeze(0)

    if _is_long(target):
        eps_sum = smooth_eps / num_classes
        eps_nll = 1. - eps_sum - smooth_eps
        likelihood = lsm.gather(dim=-1, index=target.unsqueeze(-1)).squeeze(-1)
        loss = -(eps_nll * likelihood + eps_sum * lsm.sum(-1))
    else:
        loss = -(target * lsm).sum(-1)

    if masked_indices is not None:
        loss.masked_fill_(masked_indices, 0)

    if reduction == 'sum':
        loss = loss.sum()
    elif reduction == 'mean':
        if masked_indices is None:
            loss = loss.mean()
        else:
            loss = loss.sum() / float(loss.size(0) - masked_indices.sum())

    return loss

#fine codice sorgente cross entropy + label smoothing

def onehot(indexes, N=None, ignore_index=None):
    #Crea una rappresentazione di indici con N possibili voci se N non è specificato, si adatterà all'indice massimo che appare.
    #indexes e' un tensore long di indici
    if N is None:
        N = indexes.max() + 1
    sz = list(indexes.size())
    output = indexes.new().byte().resize_(*sz, N).zero_()
    output.scatter_(-1, indexes.unsqueeze(-1), 1)
    if ignore_index is not None and ignore_index >= 0:
        output.masked_fill_(indexes.eq(ignore_index).unsqueeze(-1), 0)
    return output

def _is_long(x): #il tensore e' long? (64-bit int)
    if hasattr(x, 'data'):
        x = x.data
    return isinstance(x, torch.LongTensor) or isinstance(x, torch.cuda.LongTensor)

#mask loss
def maskNLLLoss(inp, target, mask): # calcolo la perdita basandosi sul tensore output del decoder, il tensore target e un tensore binario per la maschera
    nTotal = mask.sum()
    crossEntropy = CrossEntropyLoss(ignore_index = TRG_PAD_IDX, smooth_eps=0.20)
    loss = crossEntropy(inp, target)
    loss = loss.to(device)
    return loss, nTotal.item()

criterion = maskNLLLoss

In [27]:
def make_trg_mask(trg): # funziona come quella definita prima nella classe Transformer
      
        trg_pad_mask = (trg != TRG_PAD_IDX).unsqueeze(1).unsqueeze(2)
        trg_len = trg.shape[1]
        trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len), device = device)).bool()   
        trg_mask = trg_pad_mask & trg_sub_mask        
        return trg_mask

#allenamento del modello
def train(model, iterator, optimizer, criterion, clip):
    
    model.train() # dico al modello se siamo in modalita' allenamento o validazione, a seconda di questo la dropout si comporta diversamente
    
    n_totals = 0
    print_losses = []
    for i, batch in tqdm(enumerate(iterator), total=len(iterator)): #tqdm esegue il display della barra di progressione mentre alleniamo il modello
        loss = 0
        src = batch.Input.permute(1, 0) 
        trg = batch.Output.permute(1, 0)
        trg_mask = make_trg_mask(trg)
        optimizer.zero_grad()   #resetto la loss senza considerare quelle degli step precedenti
        
        output, _ = model(src, trg[:,:-1])                
            
        output_dim = output.shape[-1]
            
        output = output.contiguous().view(-1, output_dim)
        trg = trg[:,1:].contiguous().view(-1)
                           
        mask_loss, nTotal = criterion(output, trg, trg_mask)
        
        mask_loss.backward()   #calcola il loss gradient
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)  #per evitare gradient exploding (forza i gradient ad essere abbastanza piccoli)
        
        optimizer.step()       #aggiorna tutti i parametri basandosi sul gradient
        
        print_losses.append(mask_loss.item() * nTotal)
        n_totals += nTotal

    return sum(print_losses) / n_totals

In [28]:
#validazione del modello
def evaluate(model, iterator, criterion):
    
    model.eval()  # dico al modello se siamo in modalita' allenamento o validazione, a seconda di questo la dropout si comporta diversamente
    
    n_totals = 0
    print_losses = []
    
    with torch.no_grad():  # salto il calcolo del gradiente, ovvero non cambio i weights
        for i, batch in tqdm(enumerate(iterator), total=len(iterator)):

            src = batch.Input.permute(1, 0)
            trg = batch.Output.permute(1, 0)
            trg_mask = make_trg_mask(trg)

            output, _ = model(src, trg[:,:-1])           
            
            output_dim = output.shape[-1]
            
            output = output.contiguous().view(-1, output_dim)
            trg = trg[:,1:].contiguous().view(-1)            
            
            mask_loss, nTotal = criterion(output, trg, trg_mask)

            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
  
    return sum(print_losses) / n_totals

In [29]:
def epoch_time(start_time, end_time): # calcolo il tempo che ci vuole per ogni epoch
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [30]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [31]:
model_save_name = 'engtopy.pt'  #salvo il modello sul mio drive
path = F"/content/gdrive/My Drive/{model_save_name}"

In [32]:
N_EPOCHS = 60
CLIP = 1

best_valid_loss = float('inf')  #setto a +infinito

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_ex = []
    val_ex = []

    for i in range(train_df.shape[0]):
        try:
            ex = data.Example.fromlist([train_df.question[i], train_df.solution[i]], fields)
            train_ex.append(ex)
        except:
            pass

    for i in range(val_df.shape[0]):
        try:
            ex = data.Example.fromlist([val_df.question[i], val_df.solution[i]], fields)
            val_ex.append(ex)
        except:
            pass       

    train_data = data.Dataset(train_ex, fields)
    valid_data =  data.Dataset(val_ex, fields)

    BATCH_SIZE = 16
    #creazione dei batches in maniera ottimale ovvero con esempi simili in ogni batch tramite Pytorchtext 
    #grazie a x: len(x.Input) avro' in ogni batch degli input di lunghezza simile
    train_iterator, valid_iterator = BucketIterator.splits((train_data, valid_data), batch_size = BATCH_SIZE, 
                                                                sort_key = lambda x: len(x.Input),
                                                                sort_within_batch=True, device = device)

    train_loss = train(model, train_iterator, optimizer, criterion, CLIP) #allenamento 
    valid_loss = evaluate(model, valid_iterator, criterion)               #validazione 
    
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:   # se migliora la perdita (diminuisce) nella validazione allora salvo come migliore 
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), path)
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f}')

100%|██████████| 3/3 [00:00<00:00, 59.62it/s]


Epoch: 01 | Time: 1m 25s
	Train Loss: 4.988
	 Val. Loss: 4.206


100%|██████████| 3/3 [00:00<00:00, 59.95it/s]


Epoch: 02 | Time: 1m 23s
	Train Loss: 4.222
	 Val. Loss: 3.920


100%|██████████| 3/3 [00:00<00:00, 63.09it/s]


...

Epoch: 58 | Time: 1m 24s
	Train Loss: 2.569
	 Val. Loss: 2.944


100%|██████████| 3/3 [00:00<00:00, 62.98it/s]


Epoch: 59 | Time: 1m 24s
	Train Loss: 2.570
	 Val. Loss: 2.936


100%|██████████| 3/3 [00:00<00:00, 67.47it/s]

Epoch: 60 | Time: 1m 24s
	Train Loss: 2.563
	 Val. Loss: 2.954


In [33]:
SRC = Input
TRG = Output

In [34]:
model.load_state_dict(torch.load(path))

<All keys matched successfully>

In [35]:
#funzione per testare il funzionamento del modello
def translate_sentence(sentence, src_field, trg_field, model, device, max_len = 50000):    
    model.eval()
        
    if isinstance(sentence, str):
        nlp = spacy.load('en')
        tokens = [token.text.lower() for token in nlp(sentence)] #tokenizzazione
    else:
        tokens = [token.lower() for token in sentence]

    tokens = [src_field.init_token] + tokens + [src_field.eos_token]  #init e eos sono i tokens che segnalano l'inizio e la fine della frase
        
    src_indexes = [src_field.vocab.stoi[token] for token in tokens]   #.stoi per mappare i src tokens con indici numerabili
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)    
    src_mask = model.make_src_mask(src_tensor)
    
    with torch.no_grad():                                            # salto il calcolo del gradiente, ovvero non cambio i weights
        enc_src = model.encoder(src_tensor, src_mask)                # faccio passare l'src nell encoder

    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]       #.stoi per mappare i trg tokens con indici numerabili

    for i in range(max_len):

        trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).to(device)
        trg_mask = model.make_trg_mask(trg_tensor)
        
        with torch.no_grad():
            output, attention = model.decoder(trg_tensor, enc_src, trg_mask, src_mask)   # faccio passare trg e src codificato dentro il decoder
        
        pred_token = output.argmax(2)[:,-1].item()                  #seleziona l'indice con il valore massimo
        trg_indexes.append(pred_token)

        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break
    
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]    
    
    return trg_tokens[1:], attention

Carico il modello preaddestrato per eseguire dei test su una serie di esempi

In [36]:
model.load_state_dict(torch.load(path))

def eng_to_python(src):
  src=src.split(" ")
  translation, attention = translate_sentence(src, SRC, TRG, model, device)
  #print(f'Soluzione: \n') 
  #print(untokenize(translation[:-1]).decode('utf-8'))          #per i test

  return untokenize(translation[:-1]).decode('utf-8')


In [37]:
SRC = Input
TRG = Output

In [38]:
#bleu score
nTest = test_df.shape[0]

totalBleu = 0
skipped = 0

smoothie = SmoothingFunction().method4

for i in range(0,nTest):         #testo il modello sul dataframe di test 
  try:
    referenceTest = test_df['solution'].iloc[i].split()
    candidateTest = (eng_to_python(test_df['question'].iloc[i])).split()
    totalBleu += sentence_bleu(referenceTest,candidateTest,smoothing_function=smoothie)   
    #calcolo il bleuScore totale di tutte le domande di test, provandole una alla volta
  except:
    skipped += 1
    pass

score = totalBleu/(nTest-skipped)

print('BLEU score -> {}'.format(score))

BLEU score -> 0.16121748796741578


In [39]:
src = "function that subtract two numbers"

eng_to_python(src)

'def sub_two_numbers (num1 ,num2 ):\n    sub =num1 -num2 \n    return sub '

In [40]:
src = "program to reverse a string"

eng_to_python(src)

"\nstring ='Today is bad day'\nstring [::-1 ]"

In [41]:
src = "function to find the perimeter of a square"

eng_to_python(src)

'def square_perimeter (a ):\n    return 4 *a '